In [4]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [6]:
df = pd.read_csv('dart.csv',  sep=',')
df = df.dropna()
df

,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
...,...,...,...,...
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...


In [7]:
# Mecab 객체 생성 시 사전 경로 지정
mecab = Mecab('C:\mecab\mecab-ko-dic')

tagged_corpus_list = []

# gensim의 TaggedDocument 클래스를 사용하여 태그명(회사명)과 본문내용 저장
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['business']
    tag = row['name']
    tagged_corpus_list.append(TaggedDocument(words=mecab.morphs(text), tags=[tag]))

print('문서의 수:', len(tagged_corpus_list))

100%|██████████| 2295/2295 [01:13<00:00, 31.32it/s]

문서의 수: 2295


In [8]:
from gensim.models import doc2vec

In [9]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=20)

# 모델 저장
model.save('dart.doc2vec')

In [10]:
similar_doc = model.dv.most_similar('동화약품')
print(similar_doc)

[('종근당', 0.5618821382522583), ('삼일제약', 0.5258223414421082), ('동국제약', 0.5211503505706787), ('제일약품', 0.5121228098869324), ('비씨월드제약', 0.5042028427124023), ('삼아제약', 0.4993698000907898), ('한독', 0.49644485116004944), ('동아에스티', 0.49148866534233093), ('일양약품', 0.48961523175239563), ('영진약품', 0.4775065779685974)]


In [11]:
similar_doc = model.dv.most_similar('하이트진로')
print(similar_doc)

[('하이트진로홀딩스', 0.8474072813987732), ('보해양조', 0.5497997999191284), ('무학', 0.528270423412323), ('롯데칠성', 0.4822022020816803), ('국순당', 0.45600858330726624), ('한국팩키지', 0.42085468769073486), ('금비', 0.41181856393814087), ('롯데제과', 0.40885019302368164), ('삼광글라스', 0.40722835063934326), ('남양유업', 0.39873623847961426)]
